# github api
- https://docs.github.com/en/rest?apiVersion=2022-11-28

## 准备

In [ ]:
import requests
import json
import base64

# config in .ipython/profile_default/startup/00-github_api_config.py
# owner = '<owner>'
# repo = '<repo>'
# github_personal_access_token = '<github personal access token>'

owner = owner
repo = repo
github_personal_access_token = github_personal_access_token
domain = 'https://api.github.com'


def http_get(uri=None, **kargs):

    url = f"{domain}{uri}"
    print(f'get {url} ...')

    res = requests.get(url=url, headers={
        'Accept': 'application/vnd.github+json',
        'Authorization': f'Bearer {github_personal_access_token}',
        'X-GitHub-Api-Version': '2022-11-28'
    }, params=kargs)

    if res.status_code != 200:
        print(f"{res.status_code=}, {res.text=}")
        raise Exception(f"{res.status_code=}, {res.text=}")

    result = json.loads(res.text)
    # print(res.text)

    # print(f"{type(result)=}")
    return result

# 获取文件内容与提交信息

## get a branch 获取分支的tree_sha
- https://docs.github.com/en/rest/branches/branches?apiVersion=2022-11-28#get-a-branch

In [ ]:
# curl -L \
#   -H "Accept: application/vnd.github+json" \
#   -H "Authorization: Bearer <YOUR-TOKEN>"\
#   -H "X-GitHub-Api-Version: 2022-11-28" \
#   https://api.github.com/repos/OWNER/REPO/branches/BRANCH

def get_a_branch(branch):
    uri = f"/repos/{owner}/{repo}/branches/{branch}"
    return http_get(uri=uri)


result = get_a_branch('main')
tree_sha = result.get('commit').get('commit').get('tree').get('sha')

print(f"{tree_sha=}")

## get a tree 获取tree清单
- https://docs.github.com/en/rest/git/trees?apiVersion=2022-11-28#get-a-tree

In [ ]:
# curl -L \
#   -H "Accept: application/vnd.github+json" \
#   -H "Authorization: Bearer <YOUR-TOKEN>"\
#   -H "X-GitHub-Api-Version: 2022-11-28" \
#   https://api.github.com/repos/OWNER/REPO/git/trees/TREE_SHA
def get_a_tree(tree_sha):
    uri = f"/repos/{owner}/{repo}/git/trees/{tree_sha}?recursive=true"
    return http_get(uri)


result = get_a_tree(tree_sha)

file_list = []
for e in result.get('tree'):
    path = e.get('path')
    type = e.get('type')
    sha = e.get('sha')

    # print('\n---------------------------------')
    # print(f"{path=}, {type=}, {sha=}")

    if type == 'blob':
        file_list.append({
            'path': path,
            'sha': sha
        })

print(file_list)

## get a blob 获取文件内容
- https://docs.github.com/en/rest/git/blobs?apiVersion=2022-11-28#get-a-blob

In [ ]:
# curl -L \
#   -H "Accept: application/vnd.github+json" \
#   -H "Authorization: Bearer <YOUR-TOKEN>"\
#   -H "X-GitHub-Api-Version: 2022-11-28" \
#   https://api.github.com/repos/OWNER/REPO/git/blobs/FILE_SHA
def get_a_blob(file_sha):
    uri = f'/repos/{owner}/{repo}/git/blobs/{file_sha}'
    return http_get(uri=uri)


for file in file_list:
    result = get_a_blob(file.get('sha'))
    content = result.get('content')
    content = base64.b64decode(content).decode(encoding='utf-8')
    file.setdefault('content', content)

print(file_list)

## list commits 获取提交信息
- https://docs.github.com/en/rest/commits/commits?apiVersion=2022-11-28#list-commits

In [ ]:
# curl -L \
#   -H "Accept: application/vnd.github+json" \
#   -H "Authorization: Bearer <YOUR-TOKEN>"\
#   -H "X-GitHub-Api-Version: 2022-11-28" \
#   https://api.github.com/repos/OWNER/REPO/commits
def list_commits(path=None):
    uri = f'/repos/{owner}/{repo}/commits'
    return http_get(uri=uri, path=path)


for file in file_list:
    path = file.get('path')
    result = list_commits(path)
    result = map(lambda item: {
        'message': item.get('commit').get('message'),
        'committer': item.get('commit').get('committer')
    }, result)
    # print(list(result))
    file.setdefault('commit', list(result))


print(json.dumps(file_list, indent=4))